# Create OMMX Artifact

In [1]:
from datetime import datetime  # When the artifact is created
from dateutil.tz import tzlocal
import uuid    # To generate random tag for testing
import logging # To see the log of pushing artifact
logging.basicConfig(level=logging.INFO)

In [2]:
from ommx.artifact import Artifact, ArtifactBuilder
from ommx.testing import SingleFeasibleLPGenerator, DataType

## Ready `ommx.v1.Instance` and `ommx.v1.Solution` to be packed into artifact

In [3]:
generator = SingleFeasibleLPGenerator(3, DataType.INT)
instance = generator.get_v1_instance()
state = generator.get_v1_state()

In [4]:
# Evaluate the instance for the state
solution = instance.evaluate(state)

In [5]:
# Add annotations
instance.title = "Single feasible LP instance"
instance.created = datetime.now(tzlocal())  # Date is stored as RFC3339 format
solution.parameters = (3, "int")            # Solution can store a set of parameters as a JSON

## Build OMMX artifact

In [6]:
builder = ArtifactBuilder.for_github(
    "Jij-Inc",             # GitHub organization
    "ommx",                # Repository name
    "single_feasible_lp",  # Name of artifact
    str(uuid.uuid4())      # Tag of artifact
)

In [7]:
# Add instance to the artifact
instance_desc = builder.add_instance(instance)

In [8]:
# Add solution with the digest of instance
solution.instance = instance_desc.digest
_desc = builder.add_solution(solution)

In [9]:
# Finish building
artifact = builder.build()

In [10]:
artifact.image_name

'ghcr.io/jij-inc/ommx/single_feasible_lp:9b9083bb-9750-48bc-9847-0fd491b8bcf3'

## Push artifact to container registry

- This artifact will be pushed to <https://github.com/Jij-Inc/ommx/pkgs/container/ommx%2Fsingle_feasible_lp>
- `push` requires authentication using personal access token (PAT) or `GITHUB_TOKEN` on GitHub Actions. There are two ways for authentication:
  - Run CLI `ommx login` which stores authentication info on your machine.
  - Use environment variables `OMMX_BASIC_AUTH_{DOMAIN,USERNAME,PASSWORD}`. For example, you can push your artifact to GitHub contaier registry from GitHub Actions with following setting:

```yaml
        env:
          OMMX_BASIC_AUTH_DOMAIN: ghcr.io
          OMMX_BASIC_AUTH_USERNAME: ${{ github.actor }}
          OMMX_BASIC_AUTH_PASSWORD: ${{ github.token }}
```

In [11]:
artifact.push()

INFO:ommx.artifact:Pushing: ghcr.io/jij-inc/ommx/single_feasible_lp:9b9083bb-9750-48bc-9847-0fd491b8bcf3
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/e67c96bb-0752-4590-918a-e252806bfee2
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/eeebecfb-7884-4dab-98c3-3e8fe74921fe
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/2603c7ba-2768-406c-98de-dda48790273a
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/manifests/9b9083bb-9750-48bc-9847-0fd491b8bcf3
